In [2]:
import imgaug
import numpy as np

# Image Augmentation

In [7]:
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
import random
import PIL 

random.seed(10)
ia.seed(10)

RANDOM = random.random()

def dup_img(fname):
    arr = np.asarray(PIL.Image.open(fname)).reshape((1, 1500, 2000, 3))
    arr = np.repeat(arr, repeats=10, axis=0)
    return arr

def augment_img(images):
# Example batch of images.
# The array has shape (32, 64, 64, 3) and dtype uint8.


    seq = iaa.Sequential([
        iaa.Fliplr(0.5), # horizontal flips
        iaa.Crop(percent=(0, 0.1)), # random crops
        # Small gaussian blur with random sigma between 0 and 0.5.
        # But we only blur about 50% of all images.
        iaa.Sometimes(
            0.5,
            iaa.GaussianBlur(sigma=(0, 0.5))
        ),
        # Strengthen or weaken the contrast in each image.
        iaa.LinearContrast((0.75, 1.5)),
        # Add gaussian noise.
        # For 50% of all images, we sample the noise once per pixel.
        # For the other 50% of all images, we sample the noise per pixel AND
        # channel. This can change the color (not only brightness) of the
        # pixels.
        iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
        # Make some images brighter and some darker.
        # In 20% of all cases, we sample the multiplier once per channel,
        # which can end up changing the color of the images.
        iaa.Multiply((0.8, 1.2), per_channel=0.2),
        # Apply affine transformations to each image.
        # Scale/zoom them, translate/move them, rotate them and shear them.

        # Randomly apply affine transformations

    #     iaa.Affine(
    #         scale={"x": (0.8, 1.2 * RANDOM), "y": (0.8, 1.2* RANDOM)},
    #         translate_percent={"x": (-0.2, 0.2* RANDOM), "y": (-0.2, 0.2* RANDOM)},
    #         rotate=(-25* RANDOM, 25* RANDOM),
    #         shear=(-8* RANDOM, 8* RANDOM)
    #     )
    ], random_order=True) # apply augmenters in random order

    images_aug = seq(images=images)
    
    return images_aug

In [8]:
res = augment_img(dup_img('test.png'))

In [9]:
def grayscale_imges(rgb_img_arr):
    gray_images = []
    for rgb_img in rgb_img_arr:
        gray_img = PIL.Image.fromarray(rgb_img).convert('L')
        gray_images.append(np.array(gray_img))
    return np.array(gray_images)

In [10]:
res.shape

(10, 1500, 2000, 3)

In [11]:
gray_img = grayscale_imges(res)
import pickle

pickle.dump(gray_img, open('gray_img.pkl', 'wb'))

In [68]:
finalized_mat = gray_img.reshape((gray_img.shape[0], gray_img.shape[1] * gray_img.shape[2]))

In [69]:
finalized_mat.shape

(10, 3000000)

In [32]:
from sklearn.decomposition import PCA

In [74]:
pca = PCA(n_components=28*28)
pca.fit(finalized_mat)

ValueError: n_components=784 must be between 0 and min(n_samples, n_features)=10 with svd_solver='full'